# CosmosDB - PostGres + Azure Cognitive Search
This sample shows how to create and use search index on Azure Cognitive Search
### Prerequisite: install python libraries
- Please make sure all the libraries found in requirements.txt are installed in your python environment. 
- Rename example.env to llm.env and enter your credentials in llm.env
- Whitelist your IP to access you CosmosDB - PostGres Cluster. Add you IP in "Networking" section of your PostGRes cluster on the [Azure Portal](portal.azure.com)

# Load environment variables and keys 

In [2]:
from dotenv import dotenv_values
# specify the name of the .env file name 
env_name = "llm.env" # change to your own .env file name
config = dotenv_values(env_name)

# Connect to Cosmosdb Postgres (PG) 

In [3]:
import psycopg2
from psycopg2 import pool
host = config["host"]
dbname = config["dbname"] 
user = config["user"] 
password = config["password"] 
sslmode = config["sslmode"] 

# Build a connection string from the variables
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20,conn_string)
if (postgreSQL_pool):
    print("Connection pool created successfully")

# Use getconn() to get a connection from the connection pool
connection = postgreSQL_pool.getconn()
cursor = connection.cursor()

Connection pool created successfully


#### Load data to a table in the database
If this is the first time you are running the notebook, you need to load our sample dataset into the database first. We will create a new table "food_review_test" and load the data from the csv file.

In [6]:
import pandas as pd
import numpy as np
from psycopg2 import Error
from psycopg2 import sql

# Specify the batch size
batch_size = 25000

# Define the table name
table_name = 'food_reviews_test'

# Drop previous table of same name if one exists
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
print("Finished dropping table (if existed)")
connection.commit()

# Read data into a DataFrame
df = pd.read_csv('../DataSet/Reviews_small.csv')

# Build a connection string from the variables
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20, conn_string)
if postgreSQL_pool:
    print("Connection pool created successfully")

# Use getconn() to get a connection from the connection pool
with postgreSQL_pool.getconn() as connection:

    # Define the CREATE TABLE query
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        Id INTEGER PRIMARY KEY,
        ProductId TEXT,
        UserId TEXT,
        ProfileName TEXT,
        HelpfulnessNumerator INTEGER,
        HelpfulnessDenominator INTEGER,
        Score INTEGER,
        Time INTEGER,
        Summary TEXT,
        Text TEXT
    );
    """

    # Execute the CREATE TABLE query
    try:
        with connection.cursor() as cursor:
            cursor.execute(create_table_query)
            connection.commit()
            print(f"Table {table_name} created successfully!")
    except (Exception, Error) as e:
        print(f"Error creating table {table_name}: {e}")
        connection.rollback()

    # Assuming your DataFrame variable is called 'df'
    # Convert DataFrame to a list of tuples for bulk insertion
    records = df.to_records(index=False)
    records_list = records.tolist()

    # Convert numpy.int32 to int in each row
    records_list = [
        tuple(int(value) if isinstance(value, np.int32) else value for value in record)
        for record in records_list
    ]

    # Split the records list into batches
    batches = [records_list[i: i + batch_size] for i in range(0, len(records_list), batch_size)]

    # Iterate over each batch and perform bulk insert
    count = 0
    for batch in batches:
        count += 1
        print(f"Inserting batch {count} into the table")
        try:
            insert_query = sql.SQL(f"INSERT INTO {table_name} (Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary, Text) " \
                                   f"VALUES ({', '.join(['%s'] * len(batch[0]))})")
            
            with connection.cursor() as cursor:
                cursor.executemany(insert_query, batch)
                connection.commit()
        except (Exception, Error) as e:
            print(f"Error inserting batch into the table: {e}")
            connection.rollback()



Finished dropping table (if existed)
Connection pool created successfully
Table food_reviews_test created successfully!
Inserting batch 1 into the table


#### Example query

In [7]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()
cursor = connection.cursor()
# Execute the SELECT statement
try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

(99,)


## Retrieve data from database and store the embedding in CogSearch 
In this step, we will retrieve the id and concatenated data of desired columns from database first. Then we will use azure open ai to get the text embedding. We will then store the text embedding in azure CogSearch for the future retrieval purposes. 

#### Retrieve data from database

In [8]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()

# Execute the SELECT statement
try:
    cursor.execute(f"SELECT id, CONCAT('productid: ', productid, ' ', 'score: ', score, ' ', 'text: ', text) AS concat FROM {table_name};")
    rows = cursor.fetchall()
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

#### Create the content and generate the embedding, store in a dataframe

In [55]:
import openai
import pandas as pd

openai.api_type = config["openai_api_type"] #"azure"
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_base'] #"https://synapseml-openai.openai.azure.com/"
openai.api_version = config['openai_api_version'] 


def createEmbeddings(text):
    response = openai.Embedding.create(input=text , engine=config["openai_deployment_embedding"])
    embeddings = response['data'][0]['embedding']
    return embeddings

content_embeddings = []
idx = []
for row in rows:
    idx.append(row[0])
    content_embeddings.append(createEmbeddings(row[1]))

df = pd.DataFrame({'embeddings': content_embeddings}, index=idx) # create a dataframe with the embeddings
df

,embeddings
1,"[-0.005925467703491449, -0.0029136091470718384..."
2,"[-0.025345874950289726, -0.015591269358992577,..."
3,"[0.011405655182898045, 0.007502448279410601, -..."
4,"[0.008057798258960247, 0.006768684834241867, -..."
5,"[-0.023589162155985832, -0.009200308471918106,..."
...,...
95,"[0.011116537265479565, -0.029072990640997887, ..."
96,"[-0.0005783092346973717, -0.015766754746437073..."
97,"[0.009856229647994041, -0.009775934740900993, ..."
98,"[-0.007317356765270233, 0.0048476639203727245,..."


#### Store the embeddings in Azure Cognitive Search Vector Store

[AzureCogSearch](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search) provides a simple interface to create a vector database, store and retrieve data using vector search. You can read more about [here](https://github.com/Azure/cognitive-search-vector-pr/tree/main) more about Vector Search.

There are two steps to store data in AzureCogSearch vector database:
- First, we create the index (or schema) of the vector database
- Second, we add the chunked documents and their embeddings to the vector datastore

In [56]:
import requests
import json

# Azure Cognitive Search
cogsearch_name = config["cogsearch_name"] #TODO: fill in your cognitive search name
cogsearch_index_name = config["cogsearch_index_name"] #TODO: fill in your index name: must only contain lowercase, numbers, and dashes
cogsearch_api_key = config["cogsearch_api_key"] #TODO: fill in your api key with admin key

EMBEDDING_LENGTH = 1536

### Create search index

In [57]:
# Create Index for Cog Search with fields as id,  and contentVector
# Note the datatypes for each field below

url = f"https://{cogsearch_name}.search.windows.net/indexes/{cogsearch_index_name}?api-version=2023-07-01-Preview"
payload = json.dumps({
  "name": cogsearch_index_name,
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "filterable": True
    },
    {
      "name": "contentVector",
      "type": "Collection(Edm.Single)",
      "searchable": True,
      "retrievable": True,
      "dimensions": EMBEDDING_LENGTH,
      "vectorSearchConfiguration": "vectorConfig"
    }
  ],
  "vectorSearch": {
    "algorithmConfigurations": [
      {
        "name": "vectorConfig",
        "kind": "hnsw",
        # "hnswParameters": {
        #   "m": 4,
        #   "efConstruction": 400,
        #   "metric": "cosine"
        # }
      }
    ]
  },
  "semantic": {
    "configurations": [
      {
        "name": "my-semantic-config",
        "prioritizedFields": {
          "prioritizedContentFields": [
            {
              "fieldName": "id"
            }
          ],
        }
      }
    ]
  }
})
headers = {
  'Content-Type': 'application/json',
  'api-key': cogsearch_api_key
}

response = requests.request("PUT", url, headers=headers, data=payload)
print(response.status_code)

204


In [58]:
def batch_append_payload(df):
    """append payload for batch insertion (note: max 1000 rows per insertion) of embeddings to Cognitive Search"""
    value_list = []
    for index, row in df.iterrows():
        value_list.append(
            {
            "id": str(index),
            "contentVector": row['embeddings'],
            "@search.action": "upload"
            }
        )
    print('payload of size {}'.format(len(value_list)))
    print('start: {}'.format(value_list[0]))
    print('end: {}'.format(value_list[-1]))
    payload = json.dumps({
        "value": value_list
    })
    return payload

def BatchInsertToCogSearch(df): 
    """Batch insert of embeddings to Cognitive Search, note: column name must be 'embeddings'"""
    url = f"https://{cogsearch_name}.search.windows.net/indexes/{cogsearch_index_name}/docs/index?api-version=2023-07-01-Preview"
    payload = batch_append_payload(df)
    headers = {
    'Content-Type': 'application/json',
    'api-key': cogsearch_api_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.json())

    if response.status_code == 200 or response.status_code == 201:
        return "Success"
    else:
        return "Failure"

In [59]:
BatchInsertToCogSearch(df)

payload of size 99
start: {'id': '1', 'contentVector': [-0.005925467703491449, -0.0029136091470718384, -0.01666855439543724, 0.0030372680630534887, -0.00910672266036272, 0.029732365161180496, -0.005078488495200872, -0.046563539654016495, -0.024758901447057724, -0.03263242170214653, -0.0016473749419674277, 0.044232651591300964, -0.028783747926354408, -0.007141730282455683, -0.026872962713241577, 0.007873520255088806, 0.030626775696873665, -0.00012641167268157005, -0.01042123418301344, -0.005644270684570074, -0.03176511451601982, 0.0008241109899245203, 0.00019173495820723474, -0.008916999213397503, -0.006538681220263243, -0.00979108177125454, 0.01939244009554386, -0.01268436387181282, -0.002603614702820778, 0.012155848555266857, 0.016099384054541588, 0.008049692027270794, -0.03566799685359001, -0.004617731552571058, 0.009309997782111168, 0.004871825687587261, -0.0012780919205397367, -0.016831174492836, 0.03257821500301361, -0.031304359436035156, -0.0033387928269803524, 0.0227397028356790

'Success'

## User Asks a Question 
In this step, the code will convert the user's question to an embedding and then retieve the top K document chunks based on the users' question using the cosine similirity. Please note that other similarity metrics can also be used.

In [60]:
userQuestion = "Great Taffy"
retrieve_k = 3 # Retrieve the top 2 documents from vector database

In [61]:
# retrieve k chnuks
def retrieve_k_chunk(k, questionEmbedding):
    # Retrieve the top K entries
    url = f"https://{cogsearch_name}.search.windows.net/indexes/{cogsearch_index_name}/docs/search?api-version=2023-07-01-Preview"

    payload = json.dumps({
    "vector": {
        "value": questionEmbedding,
        "fields": "contentVector",
        "k": k
    }
    })
    headers = {
    'Content-Type': 'application/json',
    'api-key': cogsearch_api_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    output = json.loads(response.text)
    print(response.status_code)
    return output

# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = createEmbeddings(userQuestion)
output = retrieve_k_chunk(retrieve_k, questionEmbedding)

200


In [62]:
print(len(output['value']))

3


In [63]:
# Use the top k ids to retrieve the actual text from the database 
top_ids = []
for i in range(len(output['value'])):
    top_ids.append(int(output['value'][i]['id']))

print(top_ids)

[5, 8, 7]


#### Retrieve text from database

In [64]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()

format_ids = ', '.join(['%s'] * len(top_ids))

sql = f"SELECT CONCAT('productid: ', productid, ' ', 'score: ', score, ' ', 'text: ', text) AS concat FROM food_review WHERE id IN ({format_ids})"

# Execute the SELECT statement
try:
    cursor.execute(sql, top_ids)    
    top_rows = cursor.fetchall()
    for row in top_rows:
        print(row)
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")


('productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.',)
("productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!",)
('productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!',)


# OPTIONAL: Offer Response to User's Question
In order to offer a response, a user can either follow a simple prompting method as shown below or leverage more sophisticated ways used by other libraries, such as [langchain](https://python.langchain.com/en/latest/index.html).

#### Prompting directly using Azure Open AI service

In [65]:
# create a prompt template 
template = """
    context :{context}
    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: {query}
    Answer: """

In [66]:
# create the context from the top_rows
context = ""
for row in top_rows:
    context += row[0]
    context += "\n"
    
print(context)

productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!
productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!



In [67]:
print(userQuestion)
prompt = template.format(context=context, query=userQuestion)
print(prompt)

Great Taffy

    context :productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!
productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!

    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: Great Taffy
    Answer: 


In [68]:

response = openai.Completion.create(
    engine= config["openai_deployment_completion"],
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=1,
)

print("prompt: ", prompt)
print('~~~~~')
# print("response: ", response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip())
print(response['choices'][0]['text'])



prompt:  
    context :productid: B006K2ZZ7K score: 5 text: Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
productid: B006K2ZZ7K score: 5 text: This saltwater taffy had great flavors and was very soft and chewy.  Each candy was individually wrapped well.  None of the candies were stuck together, which did happen in the expensive version, Fralinger's.  Would highly recommend this candy!  I served it at a beach-themed party and everyone loved it!
productid: B006K2ZZ7K score: 5 text: This taffy is so good.  It is very soft and chewy.  The flavors are amazing.  I would definitely recommend you buying it.  Very satisfying!!

    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: Great Taffy
    Answer: 
~~~~~
 Productid: B